In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
#Split the dataset 
X = df.drop('Class',axis=1)
y = df['Class']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# Function to train the model and print classification report
def train_model(X_train, X_test, y_train, y_test):
    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Generate classification report
    report = classification_report(y_test, y_pred, zero_division=0)
    print(report)

In [5]:
train_model(X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.53      0.65        98

    accuracy                           1.00     56962
   macro avg       0.91      0.77      0.82     56962
weighted avg       1.00      1.00      1.00     56962



In [6]:
#Shape of our data
class_0 = df[df['Class'] == 0]
class_1 = df[df['Class'] == 1]
class_0.shape, class_1.shape

((284315, 31), (492, 31))

## 1. Undersampling the majority class

In [7]:
# Randomly select the sample of 492 records from majority class
class_0_sample = class_0.sample(492)
class_0_sample.shape

(492, 31)

In [8]:
new_df = pd.concat([class_0_sample,class_1],axis=0)
new_df.shape

(984, 31)

In [9]:
# Split the dataset into features and target variable
X = new_df.drop('Class',axis=1)
y = new_df['Class']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


In [10]:
train_model(X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        99
           1       0.94      0.94      0.94        98

    accuracy                           0.94       197
   macro avg       0.94      0.94      0.94       197
weighted avg       0.94      0.94      0.94       197



In [11]:
#We selected same number of records from both class using ,stratify=y at the data splitting
y_train.value_counts()

Class
1    394
0    393
Name: count, dtype: int64

## 2. Oversampling the minority class

In [12]:
#Shape of our data
class_0 = df[df['Class'] == 0]
class_1 = df[df['Class'] == 1]
class_0.shape, class_1.shape

((284315, 31), (492, 31))

In [13]:
# Randomly select the sample of 284315 records from minority class
class_1_sample = class_1.sample(284315,replace=True)
class_1_sample.shape

(284315, 31)

In [14]:
#Combine the data from class 0 and class 1
new_df = pd.concat([class_0,class_1_sample],axis=0)
new_df.shape

(568630, 31)

In [15]:
# Split the dataset into features and target variable
X = new_df.drop('Class',axis=1)
y = new_df['Class']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


In [16]:
# Train the model and print classification report
train_model(X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     56863
           1       0.95      0.88      0.91     56863

    accuracy                           0.92    113726
   macro avg       0.92      0.92      0.92    113726
weighted avg       0.92      0.92      0.92    113726



## 3. SMOTE

In [17]:
# Split the dataset into features and target variable
X = df.drop('Class',axis=1)
y = df['Class']


In [18]:
#Install necessary packages
!pip install imbalanced-learn

In [19]:
#Import SMOTE and create a instance
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')

In [20]:
y.value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [21]:
#Create records for monority class using SMOTE
X_sample, y_sample = smote.fit_resample(X,y)
y_sample.value_counts()

Class
0    284315
1    284315
Name: count, dtype: int64

In [22]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42,stratify=y_sample)
y_train.value_counts(),y_test.value_counts()

(Class
 0    227452
 1    227452
 Name: count, dtype: int64,
 Class
 1    56863
 0    56863
 Name: count, dtype: int64)

In [23]:
# Train the model and print the classification report
train_model(X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     56863
           1       0.98      0.96      0.97     56863

    accuracy                           0.97    113726
   macro avg       0.97      0.97      0.97    113726
weighted avg       0.97      0.97      0.97    113726



## 4. Using Ensemble method

In [3]:
# Split the dataset into features and target variable
X = df.drop('Class',axis=1)
y = df['Class']


In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
y_train.value_counts()

Class
0    227451
1       394
Name: count, dtype: int64

In [26]:
from sklearn.ensemble import RandomForestClassifier

mdl =  RandomForestClassifier(random_state=42)
mdl.fit(X_train,y_train)
y_preds = mdl.predict(X_test)
print(classification_report(y_test,y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.94      0.82      0.87        98

    accuracy                           1.00     56962
   macro avg       0.97      0.91      0.94     56962
weighted avg       1.00      1.00      1.00     56962

